In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer, TransformedTargetRegressor
from tqdm.notebook import tqdm
from scipy import stats
from scipy.special import exp10

In [2]:
# Read the CSV file
dat = pd.read_csv('../data/subject_level_data_for_analysis.csv')

In [3]:
# column names
ivs = ['DaysDriving',
        'Miles_n',
        "Trips", 
        "TripMinutes_n",
        "TripChains",
        "MilesPerTrip_n", 
        "MinutesPerTrip_n",
        "MilesPerChain_n",  
        "MinutesPerChain_n", 
        "Average_speed",
        "LeftTurnCount", 
        "RightToLeftTurnRatio_n",
        "TripsAMPeak", 
        "PercentTripsAMPeak_n",
        "TripsPMPeak", 
        "PercentTripsPMPeak_n",  
        "TripsAtNight", 
        "PercentTripsAtNight_n",
        "TripsLt15Miles", 
        "PercentDistLt15Miles_n",
        "TripsVgt60", 
        "PercentTripsVgt60_n", 
        "SpeedingPer1000Miles", 
        "DecelerationPer1000Miles"]

dvs = ['LIFESATISFACTION','cog','phys','fati','socialrole', 'info', 'emo', 'iso', 'ins', 'anx','dep','ang']

demos_cat = ['Site', 'GENDER', 'RACE_ETH', 'WORK', 'MARRIAGE']
demos_num = ['Age', 'EDUCATION', 'INCOME']

# convert variables to categories
categorical_cols = ['XID'] + demos_cat
for col in categorical_cols:
    if col in dat.columns:
        dat[col] = dat[col].astype('category')

# scale all dependent variables
dat[dvs] = dat[dvs].apply(lambda x: (x - x.mean()) / x.std(ddof=1))

In [4]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2658 entries, 0 to 2657
Data columns (total 47 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   XID                       2658 non-null   category
 1   n_sessions                2658 non-null   int64   
 2   n_months                  2658 non-null   int64   
 3   DaysDriving               2658 non-null   float64 
 4   Miles_n                   2658 non-null   float64 
 5   Trips                     2658 non-null   float64 
 6   TripMinutes_n             2658 non-null   float64 
 7   TripChains                2658 non-null   float64 
 8   MilesPerTrip_n            2658 non-null   float64 
 9   MinutesPerTrip_n          2658 non-null   float64 
 10  MilesPerChain_n           2658 non-null   float64 
 11  MinutesPerChain_n         2658 non-null   float64 
 12  Average_speed             2658 non-null   float64 
 13  LeftTurnCount             2658 non-null   float6

In [5]:
def nested_kfold_ridge(data, numeric_ivs, categorical_ivs, dv, id_col, alphas=None, random_state=0, n_splits_outer=10):
    """
    Nested k-fold CV for Ridge regression with mixed numeric + categorical predictors.
    """
    if alphas is None:
        alphas = np.logspace(np.log10(0.01), np.log10(500), 100)

    X = data[numeric_ivs + categorical_ivs]
    y = data[dv].values
    ids = data[id_col].values

    # Outer loop CV
    outer_cv = KFold(n_splits=n_splits_outer, shuffle=True, random_state=random_state)

    n = len(data)
    errors = np.full(n, np.nan)
    best_alphas = np.full(n, np.nan)
    folds = np.full(n, np.nan)

    for fold_idx, (train_idx, test_idx) in enumerate(tqdm(outer_cv.split(X), total=n_splits_outer, desc="Outer CV")):
        # Split into outer train/test
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # custom preprocessor for numeric and categorical variables
        preprocessor = make_column_transformer(
            (StandardScaler(), numeric_ivs),
            (OneHotEncoder(drop="first"), categorical_ivs)
            )

        # Inner CV for hyperparameter tuning
        model = make_pipeline(
            preprocessor,
            RidgeCV(alphas=alphas, scoring='neg_mean_squared_error'),
            )
        model.fit(X_train, y_train)

        # Predict on outer test fold
        y_pred = model.predict(X_test)

        # Store results
        errors[test_idx] = (y_test - y_pred) ** 2
        best_alphas[test_idx] = model.named_steps['ridgecv'].alpha_
        folds[test_idx] = fold_idx

    # Create results dataframe
    results = pd.DataFrame({
        "subject": ids,
        "fold": folds,
        "error": errors,
        "best_alpha": best_alphas
    })

    print('mean of best alphas: ', results.groupby('fold')['best_alpha'].first().mean())
    print('max of best alphas: ', results.groupby('fold')['best_alpha'].first().max())
    print('min of best alphas: ', results.groupby('fold')['best_alpha'].first().min())

    return results


# ---------- Compare Full vs Reduced ----------
def run_nestedCV(data, dv, id_col,
                 numeric_full, categorical_full,
                 numeric_reduced, categorical_reduced, alphas=None,
                 save_path=None):
    """
    Compare full model (driving + demographics) vs reduced model (demographics only).
    """
    # Reduced model (demographics only)
    res_reduced = nested_kfold_ridge(data, numeric_reduced, categorical_reduced, dv, id_col, alphas=alphas)

    # Full model (driving + demographics)
    res_full = nested_kfold_ridge(data, numeric_full, categorical_full, dv, id_col, alphas=alphas)

    # Merge by subject
    res = pd.merge(res_reduced, res_full,
                   on='subject', how='inner',
                   suffixes=('_reduced', '_full'))

    # Difference in MSE (paired)
    res['diff'] = res['error_full'] - res['error_reduced']

    # Paired one-sample t-test
    t_stat, p_value = stats.ttest_1samp(res['diff'], 0, alternative='less')

    print(f"\n=== Results for DV: {dv} ===")
    print(f"MSE (Reduced): {res['error_reduced'].mean():.4f}")
    print(f"MSE (Full):    {res['error_full'].mean():.4f}")
    print(f"Difference (Full - Reduced): {res['diff'].mean():.4f}")
    print(f"T-statistic = {t_stat:.3f}, P-value = {p_value:.4f}")

    # Optionally save
    if save_path:
        res.to_csv(f"{save_path}/{dv}.csv", index=False)

    return res

# Driving + Demographic vs. Demographic Only

In [6]:
# Life Satisfaction 
alphas = np.linspace(0.01, 120, 100)
life_satisfaction = run_nestedCV(
    data=dat, dv = 'LIFESATISFACTION', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  91.75992929292929
max of best alphas:  104.24373737373737
min of best alphas:  82.42737373737374


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  81.94256565656566
max of best alphas:  98.18363636363637
min of best alphas:  58.18696969696969

=== Results for DV: LIFESATISFACTION ===
MSE (Reduced): 0.8740
MSE (Full):    0.8374
Difference (Full - Reduced): -0.0365
T-statistic = -4.868, P-value = 0.0000


In [7]:
# Cognitive Decline
alphas = np.linspace(0.01, 10, 100)
cognitive_decline = run_nestedCV(
    data=dat, dv = 'cog', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  4.248181818181817
max of best alphas:  5.66090909090909
min of best alphas:  3.239090909090909


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  5.862727272727272
max of best alphas:  7.477272727272727
min of best alphas:  4.449999999999999

=== Results for DV: cog ===
MSE (Reduced): 0.8681
MSE (Full):    0.8703
Difference (Full - Reduced): 0.0023
T-statistic = 0.472, P-value = 0.6814


In [8]:
# Physical Decline
alphas = np.linspace(0.01, 300, 100)
frailty = run_nestedCV(
    data=dat, dv = 'phys', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  203.63957575757576
max of best alphas:  245.45636363636362
min of best alphas:  151.520101010101


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  30.009000000000004
max of best alphas:  33.34222222222222
min of best alphas:  24.251616161616163

=== Results for DV: phys ===
MSE (Reduced): 0.9416
MSE (Full):    0.8811
Difference (Full - Reduced): -0.0605
T-statistic = -5.055, P-value = 0.0000


In [9]:
# Fatigue
alphas = np.linspace(0.01, 300, 100)
fatigue = run_nestedCV(
    data=dat, dv = 'fati', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  117.88485858585857
max of best alphas:  248.48656565656563
min of best alphas:  63.64424242424242


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  64.85632323232323
max of best alphas:  78.79525252525252
min of best alphas:  45.4630303030303

=== Results for DV: fati ===
MSE (Reduced): 0.9797
MSE (Full):    0.9611
Difference (Full - Reduced): -0.0187
T-statistic = -2.947, P-value = 0.0016


In [10]:
# Social Role Constraints
alphas = np.linspace(0.01, 300, 100)
social_role = run_nestedCV(
    data=dat, dv = 'socialrole', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  191.51876767676768
max of best alphas:  242.4261616161616
min of best alphas:  96.97646464646465


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  141.82345454545458
max of best alphas:  187.88252525252523
min of best alphas:  84.85565656565657

=== Results for DV: socialrole ===
MSE (Reduced): 0.9842
MSE (Full):    0.9632
Difference (Full - Reduced): -0.0209
T-statistic = -3.136, P-value = 0.0009


In [11]:
# Informational Support
alphas = np.linspace(0.01, 50, 100)
informational_support = run_nestedCV(
    data=dat, dv = 'info', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  10.86641414141414
max of best alphas:  13.138686868686868
min of best alphas:  6.574343434343434


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  18.390161616161617
max of best alphas:  21.722828282828285
min of best alphas:  11.118888888888888

=== Results for DV: info ===
MSE (Reduced): 0.9377
MSE (Full):    0.9351
Difference (Full - Reduced): -0.0025
T-statistic = -0.462, P-value = 0.3221


In [12]:
# Emotional Support
alphas = np.linspace(0.01, 50, 100)
emotional_support = run_nestedCV(
    data=dat, dv = 'emo', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  9.351565656565658
max of best alphas:  12.633737373737373
min of best alphas:  7.0792929292929285


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  20.3089696969697
max of best alphas:  28.792121212121213
min of best alphas:  16.673333333333336

=== Results for DV: emo ===
MSE (Reduced): 0.9573
MSE (Full):    0.9509
Difference (Full - Reduced): -0.0063
T-statistic = -0.970, P-value = 0.1660


In [13]:
# Instrumental Support
alphas = np.linspace(0.01, 10, 100)
instrumental_support = run_nestedCV(
    data=dat, dv = 'ins', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  4.974727272727273
max of best alphas:  5.56
min of best alphas:  4.248181818181818


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  8.193727272727271
max of best alphas:  9.293636363636363
min of best alphas:  7.376363636363636

=== Results for DV: ins ===
MSE (Reduced): 0.8453
MSE (Full):    0.8407
Difference (Full - Reduced): -0.0046
T-statistic = -0.776, P-value = 0.2190


In [14]:
# Isolation
alphas = np.linspace(0.01, 1000, 100)
isolation = run_nestedCV(
    data=dat, dv = 'iso', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  206.06854545454547
max of best alphas:  272.73454545454547
min of best alphas:  171.72545454545454


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  368.69318181818187
max of best alphas:  464.6518181818182
min of best alphas:  262.63363636363636

=== Results for DV: iso ===
MSE (Reduced): 0.9759
MSE (Full):    0.9661
Difference (Full - Reduced): -0.0099
T-statistic = -1.841, P-value = 0.0329


In [15]:
# Depression
alphas = np.linspace(0.01, 1000, 100)
depression = run_nestedCV(
    data=dat, dv = 'dep', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  337.3803636363636
max of best alphas:  454.5509090909091
min of best alphas:  242.4318181818182


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  576.7719090909092
max of best alphas:  717.1745454545455
min of best alphas:  484.8536363636364

=== Results for DV: dep ===
MSE (Reduced): 0.9808
MSE (Full):    0.9707
Difference (Full - Reduced): -0.0100
T-statistic = -2.302, P-value = 0.0107


In [16]:
# Anxiety
alphas = np.linspace(0.01, 1500, 100)
anxiety = run_nestedCV(
    data=dat, dv = 'anx', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  454.55242424242425
max of best alphas:  530.3094949494949
min of best alphas:  378.7953535353535


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  757.580707070707
max of best alphas:  984.8519191919193
min of best alphas:  545.4609090909091

=== Results for DV: anx ===
MSE (Reduced): 0.9849
MSE (Full):    0.9792
Difference (Full - Reduced): -0.0057
T-statistic = -1.499, P-value = 0.0669


In [17]:
# Anger
alphas = np.linspace(0.01, 50, 100)
anger = run_nestedCV(
    data=dat, dv = 'ang', id_col = 'XID', numeric_full = ivs+demos_num, categorical_full = demos_cat, 
    numeric_reduced = demos_num, categorical_reduced = demos_cat, alphas = alphas, save_path='cv_errors')

Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  23.439656565656566
max of best alphas:  28.792121212121213
min of best alphas:  16.673333333333336


Outer CV:   0%|          | 0/10 [00:00<?, ?it/s]

mean of best alphas:  35.40695959595959
max of best alphas:  46.97030303030303
min of best alphas:  24.752525252525253

=== Results for DV: ang ===
MSE (Reduced): 0.9820
MSE (Full):    0.9855
Difference (Full - Reduced): 0.0036
T-statistic = 0.913, P-value = 0.8192
